In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_excel('Survey.xlsx', engine='openpyxl')
print(df.columns)

Index(['ID', 'Start time', 'Completion time', 'Email', 'Name',
       'Last modified time',
       'By proceeding with the study, I agree to all of the following statements: \nI have read and understood the above information. I understand that my participation is voluntary, and I can withdraw at ...',
       'In which sector(s) do you work? ',
       'What are your areas of core expertise? ',
       'What are your areas of core expertise? ',
       'In what region/locality do you primarily work?',
       'In what region/locality do you primarily work? (If UK selected)',
       'In what country do you primarily work? (If international selected)',
       'How would you define your career stage? '],
      dtype='object')


In [3]:
# lose the white space at begining or end of the column name
df.columns = df.columns.str.strip()

# rename most of them
df = df.rename(columns={
    'ID': 'id',
    'Start time': 'start_time',
    'Completion time': 'completion_time',
    'Email': 'email',
    'Name': 'name',
    'Last modified time': 'last_modified_time',
    'In which sector(s) do you work?': 'sectors',
    'What are your areas of core expertise?': "areas1",
    'What are your areas of core expertise? 2': "areas2",
    'In what region/locality do you primarily work?': 'country_uk',
    'In what region/locality do you primarily work? (If UK selected)': 'region_uk',
    'In what country do you primarily work? (If international selected)': 'country_international',
    'How would you define your career stage?': 'career_stage'
})

print(df.columns)

Index(['id', 'start_time', 'completion_time', 'email', 'name',
       'last_modified_time',
       'By proceeding with the study, I agree to all of the following statements: \nI have read and understood the above information. I understand that my participation is voluntary, and I can withdraw at ...',
       'sectors', 'areas1', 'What are your areas of core expertise?',
       'country_uk', 'region_uk', 'country_international', 'career_stage'],
      dtype='object')


In [4]:
# find the long consent column and rename
consent_col = [col for col in df.columns if 'statements' in col]
print(consent_col)

df = df.rename(columns={consent_col[0]: 'consent'})

print(df.columns)

['By\xa0proceeding\xa0with the study, I agree to\xa0all of\xa0the following statements:\xa0\nI have read and understood the above information.\xa0I understand that my participation is voluntary, and I can withdraw at ...']
Index(['id', 'start_time', 'completion_time', 'email', 'name',
       'last_modified_time', 'consent', 'sectors', 'areas1',
       'What are your areas of core expertise?', 'country_uk', 'region_uk',
       'country_international', 'career_stage'],
      dtype='object')


In [5]:
# find areas2 and rename
area_col = [col for col in df.columns if 'expertise' in col]
print(area_col)

df = df.rename(columns={area_col[0]: 'areas2'})

print(df.columns)

['What are your areas of core\xa0expertise?']
Index(['id', 'start_time', 'completion_time', 'email', 'name',
       'last_modified_time', 'consent', 'sectors', 'areas1', 'areas2',
       'country_uk', 'region_uk', 'country_international', 'career_stage'],
      dtype='object')


In [6]:
# Convert DataFrame to JSON
json_data = df.to_json(orient='records')
formatted_json = json.loads(json_data)
# print(formatted_json)

In [7]:
for attendee in formatted_json:

    # Processing 'sectors'
    sectors_l = attendee.get('sectors', '')
    if sectors_l and isinstance(sectors_l, str):
        sectors_list = sectors_l.split(';')
        new_sectors_list = [sector.replace("-", " ").strip().lower().capitalize() for sector in sectors_list if sector and sector.strip() != ""]
    else:
        new_sectors_list = []
    attendee['sectors'] = new_sectors_list

    # Processing 'areas1'
    areas1_str = attendee.get('areas1', '')
    if areas1_str and isinstance(areas1_str, str):
        areas1_list = areas1_str.split(';')
        new_areas1_list = [area.replace("-", " ").strip().lower().capitalize() for area in areas1_list if area and area.strip() != ""]
    else:
        new_areas1_list = []
    attendee['areas1'] = new_areas1_list

    # Processing 'areas2'
    areas2_str = attendee.get('areas2', '')
    if areas2_str and isinstance(areas2_str, str):
        areas2_list = areas2_str.split(',')
        new_areas2_list = [area.replace("-", " ").strip().lower().capitalize() for area in areas2_list if area and area.strip() != ""]
    else:
        new_areas2_list = []

    attendee['areas2'] = new_areas2_list
    attendee['areas_all'] = new_areas1_list + new_areas2_list


In [8]:
def clean_area(area):
    if area == "artificial intelligence":
        return "AI"
    elif "artificial intelligence" in area.lower():
        return "AI"
    elif "ai" in area.lower():
        return "AI"
    elif "interdisciplinary" in area.lower():
        return "Interdisciplinary"
    elif "technical" in area.lower():
        return "Technical"
    elif "philosophy" in area.lower():
        return "Philosophy"
    elif "ethics" in area.lower():
        return "Ethics"
    elif "law" in area.lower():
        return "Law"
    elif "policy" in area.lower():
        return "Policy"
    elif "machine learning" in area.lower():
        return "ML"
    elif "disinformation" in area.lower():
        return "Disinformation"
    elif "misinformation" in area.lower():
        return "Misinformation"
    elif "human computer interaction" in area.lower():
        return "HCI"
    else:
        return area

In [9]:
def clean_sector(sector):
    if "government" in sector:
        return "public"
    elif "research" in sector:
        return "academia"
    elif "third" in sector:
        return "civil society"
    elif "unemployed" in sector.lower():
        return "other"
    elif "ngo" in sector.lower():
        return "other"
    else:
        return sector

In [10]:
area_counts = {}
area_clean_counts = {}

for record in formatted_json:
    # new list
    cleaned_areas = [clean_area(area) for area in record["areas_all"]]
    record["areas_cleaned"] = list(set(cleaned_areas))
    
    # new sector
    cleaned_sectors = [clean_sector(sector) for sector in record["sectors"]]
    record["sectors_cleaned"] = list(set(cleaned_sectors))
    
    for area in record["areas_all"]:
        if area in area_counts:
            area_counts[area] += 1
        else:
            area_counts[area] = 1

    for area in record["areas_cleaned"]:
        if area in area_clean_counts:
            area_clean_counts[area] += 1
        else:
            area_clean_counts[area] = 1

result = [{"area": key, "count": value} for key, value in area_counts.items()]
result_clean = [{"area": key, "count": value} for key, value in area_clean_counts.items()]

sorted_result = sorted(result, key=lambda x: x["count"], reverse=True)
sorted_result_clean = sorted(result_clean, key=lambda x: x["count"], reverse=True)

print(sorted_result[:10])
print(sorted_result_clean[:10])

output_area = {"areas_all": sorted_result}
output_area_clean = {"areas_all_clean": sorted_result_clean}

formatted_output_area = json.dumps(output_area, indent=4)
formatted_output_area_clean = json.dumps(output_area_clean, indent=4)

# with open('output_area.json', 'w') as json_file:
#     json_file.write(formatted_output_area)
    
# with open('output_area_clean.json', 'w') as json_file:
#     json_file.write(formatted_output_area_clean)

[{'area': 'Humanities', 'count': 52}, {'area': 'Social sciences', 'count': 51}, {'area': 'Arts', 'count': 33}, {'area': 'Technical (computing and/or engineering)', 'count': 33}, {'area': 'Policy', 'count': 32}, {'area': 'Multi/interdisciplinary (describe below)', 'count': 21}, {'area': 'Ethics', 'count': 10}, {'area': 'Business', 'count': 9}, {'area': 'Law', 'count': 7}, {'area': 'Ai', 'count': 6}]
[{'area': 'Humanities', 'count': 52}, {'area': 'Social sciences', 'count': 51}, {'area': 'AI', 'count': 49}, {'area': 'Technical', 'count': 34}, {'area': 'Arts', 'count': 33}, {'area': 'Policy', 'count': 28}, {'area': 'Interdisciplinary', 'count': 22}, {'area': 'Ethics', 'count': 20}, {'area': 'Philosophy', 'count': 12}, {'area': 'Business', 'count': 9}]


In [11]:
top_12_clean = sorted_result_clean[:12]
top_12_clean_areas = [item["area"] for item in top_12_clean]

for record in formatted_json:
    record["areas_short"] = [area for area in record["areas_cleaned"] if area in top_12_clean_areas]

In [12]:
top_clean = sorted_result_clean[:30]
top_clean_areas = [item["area"] for item in top_clean]

for record in formatted_json:
    record["areas_grouped"] = [area for area in record["areas_cleaned"] if area in top_clean_areas]

In [13]:
for record in formatted_json:
    if record["career_stage"] in ["Early career", "Mid career", "Advanced career"]:
        record["career"] = [record["career_stage"]]
    else:
        record["career"] = ["Other"]

In [14]:
output_data = {"attendees": formatted_json}
formatted_output_data = json.dumps(output_data, indent=4)

# Write
with open('output_attendees.json', 'w') as json_file:
    json_file.write(formatted_output_data)